# Video Classifier Using CNN and RNN
#!dir

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('dataset/train')

label_types = os.listdir('dataset/train')
print (label_types)  

['NonViolence', 'Violence']


# Preparing Training Data

In [2]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/train' + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())


           tag                             video_name
0  NonViolence     dataset/train/NonViolence/NV_1.mp4
1  NonViolence    dataset/train/NonViolence/NV_10.mp4
2  NonViolence   dataset/train/NonViolence/NV_100.mp4
3  NonViolence  dataset/train/NonViolence/NV_1000.mp4
4  NonViolence   dataset/train/NonViolence/NV_101.mp4
           tag                        video_name
1995  Violence  dataset/train/Violence/V_995.mp4
1996  Violence  dataset/train/Violence/V_996.mp4
1997  Violence  dataset/train/Violence/V_997.mp4
1998  Violence  dataset/train/Violence/V_998.mp4
1999  Violence  dataset/train/Violence/V_999.mp4


In [3]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

# Preparing Test Data

In [4]:
dataset_path = os.listdir('dataset/test')
print(dataset_path)

room_types = os.listdir('dataset/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/test' + '/' +item) + '/' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['NonViolence', 'Violence']
Types of activities found:  2
           tag                            video_name
0  NonViolence  dataset/test/NonViolence/NV_1000.mp4
1  NonViolence   dataset/test/NonViolence/NV_986.mp4
2  NonViolence   dataset/test/NonViolence/NV_987.mp4
3  NonViolence   dataset/test/NonViolence/NV_988.mp4
4  NonViolence   dataset/test/NonViolence/NV_989.mp4
         tag                       video_name
61  Violence  dataset/test/Violence/V_995.mp4
62  Violence  dataset/test/Violence/V_996.mp4
63  Violence  dataset/test/Violence/V_997.mp4
64  Violence  dataset/test/Violence/V_998.mp4
65  Violence  dataset/test/Violence/V_999.mp4


In [5]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to c:\users\rames\appdata\local\temp\pip-req-build-52bn9u4c
  Resolved https://github.com/tensorflow/docs to commit 2e0e4a5f3521a22d0d9fa3dd832a42d1d71be283
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs 'C:\Users\rames\AppData\Local\Temp\pip-req-build-52bn9u4c'


In [6]:
pip install tensorflow

In [7]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


In [8]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

# Data preparation

In [10]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 2000
Total videos for testing: 66


,Unnamed: 0,video_name,tag
424,424,dataset/train/NonViolence/NV_480.mp4,NonViolence
1808,1808,dataset/train/Violence/V_826.mp4,Violence
1018,1018,dataset/train/Violence/V_114.mp4,Violence
935,935,dataset/train/NonViolence/NV_940.mp4,NonViolence
1985,1985,dataset/train/Violence/V_986.mp4,Violence
825,825,dataset/train/NonViolence/NV_841.mp4,NonViolence
89,89,dataset/train/NonViolence/NV_179.mp4,NonViolence
525,525,dataset/train/NonViolence/NV_571.mp4,NonViolence
1061,1061,dataset/train/Violence/V_153.mp4,Violence
1462,1462,dataset/train/Violence/V_514.mp4,Violence


# Feed the videos to a network:


In [11]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

   ### Feature Extraction

In [12]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

### Label Encoding
StringLookup layer encode the class labels as integers.

In [13]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['NonViolence', 'Violence']


array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]], dtype=int64)

Finally, we can put all the pieces together to create our data processing utility.

In [14]:
#print(train_data[0].shape)
#train_data[0]

In [15]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [16]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters

Frame features in train set: (2000, 20, 2048)
Frame masks in train set: (2000, 20)
train_labels in train set: (2000, 1)
test_labels in train set: (66, 1)


# The sequence model
Now, we can feed this data to a sequence model consisting of recurrent layers like GRU.

In [17]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
44/44 [==============================] - ETA: 0s - loss: 0.6846 - accuracy: 0.7143
Epoch 1: val_loss improved from inf to 0.73566, saving model to ./tmp\video_classifier
44/44 [==============================] - 12s 88ms/step - loss: 0.6846 - accuracy: 0.7143 - val_loss: 0.7357 - val_accuracy: 0.0000e+00
Epoch 2/30
43/44 [============================>.] - ETA: 0s - loss: 0.6693 - accuracy: 0.7137
Epoch 2: val_loss did not improve from 0.73566
44/44 [==============================] - 1s 28ms/step - loss: 0.6691 - accuracy: 0.7143 - val_loss: 0.7767 - val_accuracy: 0.0000e+00
Epoch 3/30
43/44 [============================>.] - ETA: 0s - loss: 0.6550 - accuracy: 0.7188
Epoch 3: val_loss did not improve from 0.73566
44/44 [==============================] - 1s 28ms/step - loss: 0.6560 - accuracy: 0.7143 - val_loss: 0.8166 - val_accuracy: 0.0000e+00
Epoch 4/30
42/44 [===========================>..] - ETA: 0s - loss: 0.6453 - accuracy: 0.7135
Epoch 4: val_loss did not improve from 0

3/3 [==============================] - 0s 8ms/step - loss: 0.7167 - accuracy: 0.2273
Test accuracy: 22.73%


# Inference

In [18]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

cap = cv2.VideoCapture(0)

test_video = np.random.choice(test_df["video_name"].values.tolist())
#test_video=cap
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


Test video path: dataset/test/Violence/V_985.mp4
1/1 [==============================] - 3s 3s/step
  NonViolence: 52.08%
  Violence: 47.92%


In [19]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="dataset/test/yoga/Dristi_shavasana.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")
